In [1]:
import numpy as np
import cv2
import scipy
import matplotlib.pyplot as plt
import math
import os
from Utils.ImageUtils import *
from Utils.MiscUtils import *
from Utils.GeometryUtils import *
import random
  

K = np.array([[  9.037596e+02, 0.00000000e+00,6.957519e+02],
            [  0.00000000e+00,9.019653e+02,2.242509e+02],
            [  0.00000000e+00,0.00000000e+00,1.00000000e+00]])

#Distortion Coefficients
dist = np.array([[ -3.639558e-01, 1.788651e-01, 6.029694e-04, -3.922424e-04, -5.382460e-02]])




In [2]:
folder_name = "/home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/data"
images = loadImages(folder_name)


Loading images from  /home/sakshi/courses/ENPM673/sakshi_p2/Data/Project2_Dataset2/data_1/data


In [4]:
old_right_line = None
old_left_line = None
while images:
    image = images.pop(0)
    h, w, _ = image.shape
    #undistort the image
    image_undistorted = cv2.undistort(image, K, dist)

    #remove noise
    image_blur = cv2.GaussianBlur(image_undistorted, (5,5), 0)

    #get bin
    image_bin = extractWhite(image_blur, threshold = 250)

    #roi
    image_roi, cropped_image = getROI(image_bin, 0.45) 

    #extract points
    filtered_image_roi = sobelXYThresh(image_roi)

    #get lines
    linesP = cv2.HoughLinesP(image_roi, 1, np.pi / 180, 50, None, 50, 10)

    #select left and right lines
    right_line, left_line = filterLines(linesP, old_right_line, old_left_line)
    old_right_line = right_line
    old_left_line = left_line

    # extend line

    #draw lines    
    c = extendLines(right_line,  0.45 * h,  h)
    d = extendLines(left_line,  0.45 * h,  h)
    cv2.line(image_blur, (c[0], c[1]), (c[2], c[3]), (0,random.randint(0, 255),random.randint(0,255)), 3, cv2.LINE_AA)
    cv2.line(image_blur, (d[0], d[1]), (d[2], d[3]), (0,random.randint(0, 255),random.randint(0,255)), 3, cv2.LINE_AA)

    cv2.imshow('frame', image_blur)
    cv2.waitKey(100)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [19]:
image = images[10]
h, w, _ = image.shape
#undistort the image
image_undistorted = cv2.undistort(image, K, dist)

#remove noise
image_blur = cv2.GaussianBlur(image_undistorted, (5,5), 0)

#get bin
image_bin = extractWhite(image_blur, threshold = 250)

#roi
image_roi, cropped_image = getROI(image_bin, 0.45) 

#extract points
filtered_image_roi = sobelXYThresh(image_roi)

#get lines
linesP = cv2.HoughLinesP(image_roi, 1, np.pi / 180, 50, None, 50, 10)

#select left and right lines
right_line, left_line = filterLinestest(linesP, old_right_line, old_left_line)
old_right_line = right_line
old_left_line = left_line

# extend line

#draw lines    
c = extendLines(right_line,  0.45 * h,  h)
d = extendLines(left_line,  0.45 * h,  h)
cv2.line(image_blur, (c[0], c[1]), (c[2], c[3]), (0,random.randint(0, 255),random.randint(0,255)), 3, cv2.LINE_AA)
cv2.line(image_blur, (d[0], d[1]), (d[2], d[3]), (0,random.randint(0, 255),random.randint(0,255)), 3, cv2.LINE_AA)

cv2.imshow('frame', image_blur)
cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
0.45 * h,  h

(230.4, 512)